In [31]:
import json
from collections import defaultdict
import torch
from vocab import Vocab
import os
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

## Setup

In [2]:
object_state_types = ['isToggled', 'isBroken', 'isFilledWithLiquid', 'isDirty',
                      'isUsedUp', 'isCooked', 'ObjectTemperature', 'isSliced',
                      'isOpen', 'isPickedUp', 'mass', 'parentReceptacles']
receptacle_key = 'receptacleObjectIds'

## Transformations

In [3]:
def detect_type_state_change(instance_state_change, object_instance_list_sorted, object_type_list_sorted):
    state_change = []
    
    curr_typ = object_instance_list_sorted[0].split('|')[0]
    changes = []
    for i, instance_name in enumerate(object_instance_list_sorted):
        instance_typ = instance_name.split('|')[0]
        if instance_typ != curr_typ:
            state_change.append(True in changes)
            changes = []
            curr_typ = instance_typ
        changes.append(instance_state_change[i])
            
    # don't forget about last typ!
    state_change.append(int(True in changes))
    assert len(state_change) == len(object_type_list_sorted)
    return state_change

def detect_type_visibility(instance_visible, object_instance_list_sorted, object_type_list_sorted):
    type_visible = []
    
    curr_typ = object_instance_list_sorted[0].split('|')[0]
    visible = []
    for i, instance_name in enumerate(object_instance_list_sorted):
        instance_typ = instance_name.split('|')[0]
        if instance_typ != curr_typ:
            type_visible.append(True in visible)
            visible = []
            curr_typ = instance_typ
        visible.append(instance_visible[i])
            
    # don't forget about last typ!
    type_visible.append(int(True in visible))
    assert len(type_visible) == len(object_type_list_sorted)
    return type_visible

In [4]:
def shift_states(saved_states):
    # shift all the states to the last time step
    shifted_states = [{} for _ in range(len(saved_states))]

    for t, state in enumerate(saved_states):
        new_state = {k:v for k,v in state.items() if k != 'objects_metadata'}
        if t < len(saved_states) - 1:
            new_state['objects_metadata'] = saved_states[t+1]['objects_metadata']
        elif t == len(saved_states) - 1: # <<stop>> action
            new_state['objects_metadata'] = saved_states[t]['objects_metadata']
        shifted_states[t] = new_state
        
    return shifted_states

In [5]:
def get_receptacle_list(state, object_instance_list_sorted):
    '''make receptacleId list in the same order as object Id List'''
    # positions corresponds to end of task
    receptacleId_lookup = defaultdict(lambda: 'None')
    for o in state['objects_metadata']:
        if o['parentReceptacles']:
            receptacleId_lookup[o['objectId'].lower()] = o['parentReceptacles'][0].lower()
        else:
            receptacleId_lookup[o['objectId'].lower()] = 'none'

    receptacleId_list = []
    receptacleType_list = []    
    for i, instance_name in enumerate(object_instance_list_sorted):
        receptacleId = receptacleId_lookup[instance_name]
        receptacleId_list.append(receptacleId)
        receptacleType_list.append(receptacleId.split('|')[0])
    
    assert len(receptacleId_list) == len(object_instance_list_sorted)
    return receptacleId_list, receptacleType_list

In [6]:
def get_object_states(obj, required_state_types):
    o_states = {}
    for state_typ in required_state_types:
        if state_typ in obj.keys():
            o_states[state_typ] = obj[state_typ]
        else:
            o_states[state_typ] = None
    return o_states

def detect_state_change(last_obj_states, curr_obj_states, objectId_list):
    state_change = []
    for obj_Id in objectId_list:
        if (not obj_Id in last_obj_states) and (not obj_Id in curr_obj_states):
            state_change.append(False)
        elif not obj_Id in last_obj_states:
            state_change.append(True)
        elif not obj_Id in curr_obj_states:
            raise Exception('Objects should always appear in next time step.')
        else:
            if last_obj_states[obj_Id] == curr_obj_states[obj_Id]:
                state_change.append(False)
            else:
                state_change.append(True)
    assert len(state_change) == len(objectId_list)
    return state_change

def detect_receptacle_change(last_obj_receptacles, curr_obj_receptacles, objectId_list):
    receptacle_change = []
    for obj_Id in objectId_list:
        # TODO
        if (not obj_Id in last_obj_receptacles) and (not obj_Id in curr_obj_receptacles):
            receptacle_change.append(False)
        elif not obj_Id in last_obj_receptacles:
            receptacle_change.append(True)
        elif not obj_Id in curr_obj_receptacles:
            raise Exception('Objects should always appear in next time step.')
        else:
            if last_obj_receptacles[obj_Id] == curr_obj_receptacles[obj_Id]:
                receptacle_change.append(False)
            else:
                receptacle_change.append(True)
    assert len(receptacle_change) == len(objectId_list)
    return receptacle_change

In [7]:
def get_obj_distance(state, object_instance_list_sorted):
    dist_lookup = defaultdict(lambda: 1e6)
    for o in state['objects_metadata']:
        dist_lookup[o['objectId'].lower()] = o['distance']
    dist_list = []
    for i, instance_name in enumerate(object_instance_list_sorted):
        dist_list.append(dist_lookup[instance_name])
    return dist_list

In [8]:
def extract_states_for_model(shifted_states, saved_states, root):

    objectId_list = sorted([o['objectId'].lower() for o in shifted_states[-1]['objects_metadata']])
    objectTyp_list = sorted(set([o.split('|')[0].lower() for o in objectId_list]))
    num_subgoals = len(set([s['subgoal_step'] for s in shifted_states]))

    try:
        feat = {
            'subgoal': [[] for _ in range(num_subgoals)],
#             'subgoal_i': [[] for _ in range(num_subgoals)],
#             'subgoal_t': [[] for _ in range(num_subgoals)],
#             'overall_t': [[] for _ in range(num_subgoals)],

            'objectInstanceList': objectId_list,
            'objectInstanceList_TypeNum': object_vocab['object_type'].word2index([o.split('|')[0] for o in objectId_list], train=True),
            'objectReceptacleList': [[] for _ in range(num_subgoals)], # [subgoal_i][subgoal timestep] = [receptacle Id for ob in objectId_list]
            'objectReceptacleList_TypeNum': [[] for _ in range(num_subgoals)],
            'instance_visibile': [[] for _ in range(num_subgoals)], # [subgoal_i][subgoal timestep] = [T/F for ob in objectId_list]
            'instance_state_change': [[] for _ in range(num_subgoals)],
            'instance_state_change_since_last_subgoal': [[] for _ in range(num_subgoals)],
            'instance_distance': [[] for _ in range(num_subgoals)],
            'instance_receptacle_change': [[] for _ in range(num_subgoals)],
            'instance_receptacle_change_since_last_subgoal': [[] for _ in range(num_subgoals)],
            'objectTypeList': objectTyp_list,
            'objectTypeList_TypeNum': object_vocab['object_type'].word2index(objectTyp_list, train=True),
            'type_visibile': [[] for _ in range(num_subgoals)], # [subgoal_i][subgoal timestep] = [T/F for ob in objectTyp_list]
            'type_state_change': [[] for _ in range(num_subgoals)],
            'type_state_change_since_last_subgoal': [[] for _ in range(num_subgoals)],
        }
    except:
        import pdb; pdb.set_trace()

    last_obj_states = {obj['objectId'].lower():get_object_states(obj, object_state_types) for obj in saved_states[0]['objects_metadata']}
    last_subgoal_obj_states = None
    
    last_obj_receptacles = {obj['objectId'].lower():obj['parentReceptacles'] for obj in saved_states[0]['objects_metadata']}
    last_subgoal_obj_receptacles = None
    
    for t, state in enumerate(shifted_states):

        subgoal_i = state['subgoal_step']
        subgoal = state['subgoal']

        if state['new_subgoal']:
            subgoal_t = 0
            last_subgoal_obj_states = last_obj_states
            last_subgoal_obj_receptacles = last_obj_receptacles

        feat['subgoal'][subgoal_i].append(subgoal)
#         feat['subgoal_i'][subgoal_i].append(subgoal_i)
#         feat['subgoal_t'][subgoal_i].append(subgoal_t)
#         feat['overall_t'][subgoal_i].append(t)

        # get receptacles identity
        receptacle_list, receptacle_type_list = get_receptacle_list(state, objectId_list)
        receptacle_type_num = object_vocab['object_type'].word2index(receptacle_type_list, train=True)
        feat['objectReceptacleList'][subgoal_i].append(receptacle_list)
        feat['objectReceptacleList_TypeNum'][subgoal_i].append(receptacle_type_num)
        
        # get state change
        curr_obj_states = {obj['objectId'].lower():get_object_states(obj, object_state_types) for obj in state['objects_metadata']}
        
        # list same order as objectId_list
        state_change = detect_state_change(last_obj_states, curr_obj_states, objectId_list)
        state_change_since_last_subgoal = detect_state_change(last_subgoal_obj_states, curr_obj_states, objectId_list)
        last_obj_states = curr_obj_states
        
        feat['instance_state_change'][subgoal_i].append(state_change)
        feat['instance_state_change_since_last_subgoal'][subgoal_i].append(state_change_since_last_subgoal)
    
        # get instance distance
        obj_dist = get_obj_distance(state, objectId_list)
        
        feat['instance_distance'][subgoal_i].append(obj_dist)
        
        # get receptacles state
        curr_obj_receptacles = {obj['objectId'].lower():obj['parentReceptacles'] for obj in state['objects_metadata']}
        
        # get receptacle change
        receptacle_change = detect_receptacle_change(last_obj_receptacles, curr_obj_receptacles, objectId_list)
        receptacle_change_since_last_subgoal = detect_receptacle_change(last_subgoal_obj_receptacles, curr_obj_receptacles, objectId_list)
        last_obj_receptacles = curr_obj_receptacles
        
        feat['instance_receptacle_change'][subgoal_i].append(receptacle_change)
        feat['instance_receptacle_change_since_last_subgoal'][subgoal_i].append(receptacle_change_since_last_subgoal)
    
        # get visibility
        visible = [False for _ in objectId_list]
        for ob in state['objects_metadata']:
            pos = objectId_list.index(ob['objectId'].lower())
            visible[pos] = ob['visible']
        feat['instance_visibile'][subgoal_i].append(visible)
        
        # get type state change
        type_state_change = detect_type_state_change(state_change, objectId_list, objectTyp_list)
        feat['type_state_change'][subgoal_i].append(type_state_change)
        type_state_change_since_last_subgoal = detect_type_state_change(state_change_since_last_subgoal, objectId_list, objectTyp_list)
        feat['type_state_change_since_last_subgoal'][subgoal_i].append(type_state_change_since_last_subgoal)

        # get type visibility
        type_visible = detect_type_visibility(visible, objectId_list, objectTyp_list)
        feat['type_visibile'][subgoal_i].append(type_visible)

        subgoal_t += 1
        
    return feat

## Args

In [32]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
args.data_root = '/root/data_alfred/'
args.data = '/root/data_alfred/unlabeled_12k_20201206/seen/'
args.splits = '/root/data_alfred/splits/debug_20201210.json'

In [39]:
object_vocab = torch.load(os.path.join(args.data, 'objects_20200522.object_vocab'))
object_vocab

{'object_type': Vocab(111)}

## TRAIN

In [33]:
with open(args.splits, 'r') as f:
    success_outpaths = json.load(f)['augmentation']
    
len(success_outpaths)

with open(args.splits.replace('.json', '_failed.json'), 'r') as f:
    failed_roots = json.load(f)['augmentation']
    
print('success paths = ', len(success_outpaths))
print('failed paths = ', len(failed_roots))

success paths =  198
failed paths =  2


In [34]:
success_outpaths[0]

{'task': 'pick_cool_then_place_in_recep-Tomato-None-SinkBasin-21/trial_T20190910_135147_740373',
 'repeat_idx': 0,
 'full_traj_success': True,
 'collected_subgoals': 6}

In [46]:
extract_feat_dum = []
extract_feat_outpaths = []

for task in success_outpaths:
    
    metadata_outpath = os.path.join(args.data, task['task'], 'metadata_states.json')
    with open(metadata_outpath, 'r') as f:
        saved_states = json.load(f)
    
    root = os.path.join(args.data, task['task'])
    
    shifted_states = shift_states(saved_states)
    extract_feat = extract_states_for_model(shifted_states, saved_states, root)
    extract_feat['metadata_path'] = metadata_outpath
    extract_feat['root'] = root
    
    extract_feat_outpath = metadata_outpath.replace('/metadata_states.json', '/extracted_feature_states.json')
    with open(extract_feat_outpath, 'w') as f:
        json.dump(extract_feat ,f)
    
    extract_feat_outpaths.append(extract_feat_outpath)
    extract_feat_dum.append(extract_feat)
    
print(len(extract_feat_dum))

198


In [45]:
extract_feat_outpaths

['/root/data_alfred/unlabeled_12k_20201206/seen/pick_cool_then_place_in_recep-Tomato-None-SinkBasin-21/trial_T20190910_135147_740373/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_heat_then_place_in_recep-Mug-None-SinkBasin-18/trial_T20190910_113852_697425/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_clean_then_place_in_recep-Spoon-None-CounterTop-11/trial_T20190913_132015_402664/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_with_movable_recep-CellPhone-Plate-Shelf-211/trial_T20190909_182029_502585/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_clean_then_place_in_recep-Lettuce-None-DiningTable-23/trial_T20190912_150010_333678/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_two_obj_and_place-RemoteControl-None-SideTable-220/trial_T20190911_203801_914153/extracted_feature_states.json',
 '

In [16]:
date = '20201227'
split_name = 'augmentation'
with open('/root/data_alfred/splits/extract_feat_states_{}_{}_notebook_outpaths.json'.format(date, split_name), 'w') as f:
    json.dump(extract_feat_outpaths, f)

## Save Vocab

In [31]:
date = '20200522'
vocab_dout_path = os.path.join('/root/data_alfred/json_feat_2.1.0', '{}_{}.vocab'.format('objects', date))
torch.save(object_vocab, vocab_dout_path)

In [29]:
object_vocab['object_type']

Vocab(111)

In [27]:
vocab_dout_path

'/root/data_alfred/json_feat_2.1.0/objects, date_date.vocab'

In [30]:
vocab_dout_path

'/root/data_alfred/json_feat_2.1.0/objects_date.vocab'

In [32]:
vocab_dout_path

'/root/data_alfred/json_feat_2.1.0/objects_20200522.vocab'

In [33]:
dum_vocab = torch.load('/root/data_alfred/json_feat_2.1.0/objects_20200522.vocab')
dum_vocab

{'object_type': Vocab(111)}